<a href="https://colab.research.google.com/github/TejaPravs/Homework2/blob/main/sentiment_analysis_using_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Roberta for Sentiment Analysis





### Introduction

In this tutorial I will be fine tuning a roberta model for the **Sentiment Analysis** problem.

#### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Importing Python Libraries and preparing the environment](#section01)
2. [Importing and Pre-Processing the domain data](#section02)
3. [Preparing the Dataset and Dataloader](#section03)
4. [Creating the Neural Network for Fine Tuning](#section04)
5. [Fine Tuning the Model](#section05)
6. [Validating the Model Performance](#section06)
7. [Saving the model and artifacts for Inference in Future](#section07)

#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

 - Data:
	 - I will be using the dataset available at [Kaggle Competition](https://www.kaggle.com/c/movie-review-sentiment-analysis-kernels-only)
	 - I will be referring only to the first csv file from the data dump: `train.tsv`

 - Language Model Used:
	 - The RoBERTa model was proposed in RoBERTa: A Robustly Optimized BERT Pretraining Approach by Yinhan Liu, Myle Ott, Naman Goyal, Jingfei Du, Mandar Joshi, Danqi Chen, Omer Levy, Mike Lewis, Luke Zettlemoyer, Veselin Stoyanov. It is based on Google’s BERT model released in 2018.
	 - [Blog-Post](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/)
	 - [Research Paper](https://arxiv.org/pdf/1907.11692)
     - [Documentation for python](https://huggingface.co/transformers/model_doc/roberta.html)


 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - GPU enabled setup

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* tqdm
* sklearn
* Robert Model and Tokenizer

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU.

In [4]:
!pip install transformers
import transformers
print(transformers.__version__)

4.46.2


In [5]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers

import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [6]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
train = pd.read_csv('/content/movies_with_tone_labels.tsv', delimiter='\t')
train = train.rename(columns = {'Emotional_tones': 'emotional_tones'})

In [8]:
train.shape

(10000, 3)

In [9]:
train.head()

,original_title,overview,emotional_tones
0,All Tomorrow's Parties,A kaleidoscopic journey into the parallel musi...,"adventurous, intriguing, playful, optimistic"
1,Troy,"In year 1250 B.C. during the late Bronze age, ...","adventurous, intriguing, tense, melancholic"
2,Varieté,The murderer “Boss” Huller – after having spen...,"mysterious, tense, intriguing, melancholic"
3,Silenced,Only 11 Americans have ever been charged under...,"provocative, inspiring, empathetic, intriguing"
4,Alexander Graham Bell: The Sound and the Silence,True story of Alexander Graham Bell's inventio...,"inspiring, adventurous, empathetic, innovative"


In [10]:
train.describe()

,original_title,overview,emotional_tones
count,10000,10000,10000
unique,9864,9965,3425
top,Wuthering Heights,No overview found.,"mysterious, tense, intriguing, empathetic"
freq,4,31,171


In [11]:
# Assuming the emotional_tones column contains error messages for invalid labels
invalid_phrases = [
    "I'm sorry",
    "It looks like",
    "It seems",
    "Please provide",
    "Sure!"
]

# Keeping only the valid rows
train = train[~train['emotional_tones'].str.contains('|'.join(invalid_phrases), na=False)]

# Check the shape of the cleaned dataset
print(train.shape)

# Preview the cleaned dataset
print(train.head())

(9994, 3)
                                     original_title  \
0                            All Tomorrow's Parties   
1                                              Troy   
2                                           Varieté   
3                                          Silenced   
4  Alexander Graham Bell: The Sound and the Silence   

                                            overview  \
0  A kaleidoscopic journey into the parallel musi...   
1  In year 1250 B.C. during the late Bronze age, ...   
2  The murderer “Boss” Huller – after having spen...   
3  Only 11 Americans have ever been charged under...   
4  True story of Alexander Graham Bell's inventio...   

                                  emotional_tones  
0    adventurous, intriguing, playful, optimistic  
1     adventurous, intriguing, tense, melancholic  
2      mysterious, tense, intriguing, melancholic  
3  provocative, inspiring, empathetic, intriguing  
4  inspiring, adventurous, empathetic, innovative  


In [12]:
valid_tones = [
    "humor", "inspiring", "playful", "sarcastic", "empathetic",
    "mysterious", "adventurous", "tense", "innovative", "rivalry",
    "provocative", "quirky", "optimistic", "melancholic",
    "lovey-dovey", "intriguing"
]

# defining function to clean and filter tones
def filter_tones(tones):
    return ", ".join(tone.strip() for tone in tones.split(", ") if tone.strip() in valid_tones)

# Clean and filter the 'emotional_tones' column
train['emotional_tones'] = train['emotional_tones'].apply(filter_tones)

# Extract and sort unique tones
unique_cleaned_tones = sorted(
    set(tone for tones in train['emotional_tones'].str.split(", ") for tone in tones if tone.strip())
)

# Display the sorted cleaned tones
print(unique_cleaned_tones)


['adventurous', 'empathetic', 'humor', 'innovative', 'inspiring', 'intriguing', 'lovey-dovey', 'melancholic', 'mysterious', 'optimistic', 'playful', 'provocative', 'quirky', 'rivalry', 'sarcastic', 'tense']


In [13]:
# Remove malformed or misspelled tones
def clean_tones(tones):
    valid = [tone for tone in tones.split(", ") if tone.strip() in valid_tones]
    return ", ".join(valid)

# Apply cleaning
train['emotional_tones'] = train['emotional_tones'].apply(clean_tones)


In [14]:
train['emotional_tones'] = train['emotional_tones'].str.strip()  # Remove leading/trailing spaces
train = train[train['emotional_tones'] != '']

In [15]:
tones = ['adventurous', 'empathetic', 'humor', 'innovative', 'inspiring',
         'intriguing', 'lovey-dovey', 'melancholic', 'mysterious',
         'optimistic', 'playful', 'provocative', 'quirky',
         'rivalry', 'sarcastic', 'tense']
tone_to_num = {}
for idx, tone in enumerate(tones):
    tone_to_num[tone] = idx
print(tone_to_num)

{'adventurous': 0, 'empathetic': 1, 'humor': 2, 'innovative': 3, 'inspiring': 4, 'intriguing': 5, 'lovey-dovey': 6, 'melancholic': 7, 'mysterious': 8, 'optimistic': 9, 'playful': 10, 'provocative': 11, 'quirky': 12, 'rivalry': 13, 'sarcastic': 14, 'tense': 15}


In [16]:
# Assuming you have a tokenizer object already defined
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
def token_count(text, tokenizer):
    return len(tokenizer.encode(text, add_special_tokens=True, truncation = True))

# Adding a new column with the token count
train['token_count'] = train['overview'].apply(lambda x: token_count(x, tokenizer))

# Filter rows that has smaller descriptions less than 50 words
train = train[train['token_count'] >= 50]

# Dropping the temporary 'token_count' column if needed
train = train.drop(columns=['token_count'])

# Check the shape of the filtered dataset
print(train.shape)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

(6126, 3)


In [17]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

<a id='section03'></a>
### Preparing the Dataset and Dataloader

I will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. I will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *SentimentData* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the Roberta model for training.
- I am using the Roberta tokenizer to tokenize the data in the `TITLE` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline.
- The *SentimentData* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [18]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, tone_to_num):
        self.tokenizer = tokenizer
        self.tone_to_num = tone_to_num
        self.data = dataframe
        self.text = dataframe.overview
        self.targets = dataframe.emotional_tones
        self.max_len = 256

    def __len__(self):
        return len(self.text)


    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
# inputs is a dictionary that contains input_ids, attention_mask, token_type_ids
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length= 256,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
# input_ids are tokenised representation of input texts(overview) based on the values/ids given to them in them in the tokeniser library
        ids = inputs['input_ids']
# attention mask uses binary variables to represent  actual tokens (1) and padded tokens with (0)
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']
# token type ids divides/differentiates sentences/texts into different segments specially for question-answering tasks
        tone_labels = [self.tone_to_num.get(tone_str, None) for tone_str in self.targets[index].split(", ")]
        tone_labels = [tl for tl in tone_labels if tl is not None]  # Filter out invalid tones
        bce_labels = [0] * 16
        for tl in [tl for tl in tone_labels if tl is not None]:
            bce_labels[tl] = 1

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(bce_labels, dtype=torch.long)
        }

In [19]:
train_size = 0.8
train_data=train.sample(frac=train_size,random_state=200)
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN, tone_to_num)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN, tone_to_num)

FULL Dataset: (6126, 3)
TRAIN Dataset: (4901, 3)
TEST Dataset: (1225, 3)


In [20]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`.
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs.
 - The data will be fed to the Roberta Language model as defined in the dataset.
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [21]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 16)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [22]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process.

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a loss of 0.8141926634122427.

In [23]:
# Creating the loss function and optimizer
loss_function = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


In [24]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct


In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix

def valid(model, testing_loader):
    model.eval()
    tr_loss = 0
    epoch_acc = 0
    nb_tr_steps = 0

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for step, data in enumerate(testing_loader, 1):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float32)

            logits = model(ids, mask, token_type_ids)
            loss = loss_function(logits, targets)
            tr_loss += loss.item()

            probs = torch.sigmoid(logits)
            preds = (probs >= 0.35).float()
            batch_acc = (preds == targets).sum().item() / targets.numel()
            epoch_acc += batch_acc
            nb_tr_steps += 1

            # Collect predictions and targets for metrics
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    # Compute metrics
    valid_loss = tr_loss / nb_tr_steps
    valid_acc = epoch_acc / nb_tr_steps

    # Multi-label metrics
    precision = precision_score(all_targets, all_preds, average='samples', zero_division=1)
    recall = recall_score(all_targets, all_preds, average='samples', zero_division=1)
    f1 = f1_score(all_targets, all_preds, average='samples', zero_division=1)
    cm = multilabel_confusion_matrix(all_targets, all_preds)

    # Print results
    print(f"Validation Loss: {valid_loss:.4f}")
    print(f"Validation Accuracy: {valid_acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix (per label):")
    print(cm)

    return valid_acc, valid_loss



In [34]:
for epoch in range(3):
    print(f"Epoch {epoch + 1}/3")
    tr_loss = 0
    epoch_acc = 0
    nb_tr_steps = 0

    model.train()
    for step, data in enumerate(training_loader, 1):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float32)

        logits = model(ids, mask, token_type_ids)
        loss = loss_function(logits, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update training metrics
        tr_loss += loss.item()
        probs = torch.sigmoid(logits.detach().cpu())
        preds = (probs >= 0.35).float()
        batch_acc = (preds == targets.detach().cpu()).sum().item() / targets.numel()
        epoch_acc += batch_acc
        nb_tr_steps += 1

        # Log metrics every 5000 steps
        if step % 5000 == 0:
            train_loss = tr_loss / nb_tr_steps
            train_acc = epoch_acc / nb_tr_steps
            print(f"Training Loss per {step} steps: {train_loss:.4f}")
            print(f"Training Accuracy per {step} steps: {train_acc:.4f}")

    # Epoch metrics
    epoch_train_loss = tr_loss / nb_tr_steps
    epoch_train_acc = epoch_acc / nb_tr_steps
    print(f"Training Loss Epoch: {epoch_train_loss:.4f}")
    print(f"Training Accuracy Epoch: {epoch_train_acc:.4f}")

    # Validation after each epoch
    valid_acc, valid_loss = valid(model, testing_loader)
    print(f"Validation Loss Epoch {epoch + 1}: {valid_loss:.4f}")
    print(f"Validation Accuracy Epoch {epoch + 1}: {valid_acc:.4f}")
    print("=" * 50)


Epoch 1/3


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss Epoch: 0.3103
Training Accuracy Epoch: 0.8529
Validation Loss: 0.3035
Validation Accuracy: 0.8537
Precision: 0.6794
Recall: 0.7697
F1 Score: 0.7139
Confusion Matrix (per label):
[[[ 372  116]
  [ 182  555]]

 [[ 149  302]
  [  69  705]]

 [[ 887  135]
  [  54  149]]

 [[1211    0]
  [  14    0]]

 [[ 801  101]
  [ 110  213]]

 [[ 123  386]
  [  27  689]]

 [[1198    0]
  [  27    0]]

 [[ 714  139]
  [ 105  267]]

 [[ 790  138]
  [  72  225]]

 [[1079   35]
  [  76   35]]

 [[ 992   86]
  [  64   83]]

 [[ 997   59]
  [ 101   68]]

 [[ 893  151]
  [  62  119]]

 [[1207    0]
  [  18    0]]

 [[1206    0]
  [  19    0]]

 [[ 482  139]
  [  78  526]]]
Validation Loss Epoch 1: 0.3035
Validation Accuracy Epoch 1: 0.8537
Epoch 2/3


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss Epoch: 0.2714
Training Accuracy Epoch: 0.8757
Validation Loss: 0.2904
Validation Accuracy: 0.8651
Precision: 0.7023
Recall: 0.7829
F1 Score: 0.7320
Confusion Matrix (per label):
[[[ 322  166]
  [ 108  629]]

 [[ 146  305]
  [  42  732]]

 [[ 980   42]
  [  92  111]]

 [[1211    0]
  [  14    0]]

 [[ 768  134]
  [  87  236]]

 [[ 136  373]
  [  26  690]]

 [[1198    0]
  [  27    0]]

 [[ 705  148]
  [  92  280]]

 [[ 820  108]
  [  82  215]]

 [[1077   37]
  [  65   46]]

 [[1029   49]
  [  71   76]]

 [[ 963   93]
  [  82   87]]

 [[ 977   67]
  [ 103   78]]

 [[1207    0]
  [  18    0]]

 [[1206    0]
  [  19    0]]

 [[ 516  105]
  [  92  512]]]
Validation Loss Epoch 2: 0.2904
Validation Accuracy Epoch 2: 0.8651
Epoch 3/3


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss Epoch: 0.2406
Training Accuracy Epoch: 0.8924
Validation Loss: 0.2943
Validation Accuracy: 0.8653
Precision: 0.7035
Recall: 0.7821
F1 Score: 0.7327
Confusion Matrix (per label):
[[[ 353  135]
  [ 131  606]]

 [[ 138  313]
  [  35  739]]

 [[ 966   56]
  [  77  126]]

 [[1211    0]
  [  14    0]]

 [[ 720  182]
  [  71  252]]

 [[ 180  329]
  [  53  663]]

 [[1198    0]
  [  27    0]]

 [[ 669  184]
  [  66  306]]

 [[ 819  109]
  [  80  217]]

 [[1064   50]
  [  70   41]]

 [[1029   49]
  [  66   81]]

 [[1004   52]
  [  96   73]]

 [[ 976   68]
  [  94   87]]

 [[1207    0]
  [  18    0]]

 [[1206    0]
  [  19    0]]

 [[ 528   93]
  [ 106  498]]]
Validation Loss Epoch 3: 0.2943
Validation Accuracy Epoch 3: 0.8653


In [103]:
outputs

tensor([[-5.4717e-02, -1.3106e-02, -7.4805e-02, -1.7144e-01, -2.3141e-03,
          6.0220e-02, -1.0050e-01, -2.2928e-02,  6.7549e-02,  1.4701e-01,
          1.6569e-02,  1.5849e-01, -2.1958e-01,  1.0397e-01,  2.6452e-01,
          1.6763e-01],
        [-7.9450e-02, -4.0141e-02, -1.8531e-01, -1.2258e-01,  6.1988e-02,
          9.9231e-03, -6.2996e-02,  4.9996e-02, -9.4893e-02,  1.8278e-01,
         -7.9654e-02,  9.4997e-02, -1.0181e-01,  1.5750e-01,  3.0294e-01,
          9.1920e-02],
        [-7.3177e-02, -5.0926e-02, -2.7425e-02, -8.7990e-02,  3.1822e-02,
          6.0473e-02, -5.0405e-02,  4.2614e-02,  4.1726e-02,  2.8491e-01,
          1.4312e-01,  1.1740e-01, -1.2281e-01,  1.3756e-01,  2.0290e-01,
          1.0004e-01],
        [-1.4649e-01, -6.0954e-02, -7.1961e-02, -2.6874e-02,  8.6942e-02,
          8.6437e-02, -3.0898e-02,  3.7832e-02,  9.3776e-02,  2.4331e-01,
         -2.9698e-02,  8.9155e-02, -2.2387e-01,  1.6813e-01,  1.7661e-01,
          1.9179e-01],
        [-1.6000e-02

In [104]:
targets

tensor([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data.

This unseen data is the 20% of `train.tsv` which was seperated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model.

As you can see the model is predicting the correct category of a given sample to a 69.47% accuracy which can further be improved by training more.

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model.

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')